In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#prova

In [5]:
import utils.data as data

train, test = data.get_train_test_split(0.1, 42)
dataset = data.Marconi100Dataset(test)

100%|██████████| 25/25 [00:06<00:00,  3.94it/s]


In [7]:
df, lab = dataset[0]

In [8]:
df.describe()

,avg:ambient,var:ambient,max:ambient,min:ambient,avg:dimm0_temp,var:dimm0_temp,max:dimm0_temp,min:dimm0_temp,avg:dimm10_temp,var:dimm10_temp,...,max:proc_total,min:proc_total,avg:swap_free,var:swap_free,max:swap_free,min:swap_free,avg:swap_total,var:swap_total,max:swap_total,min:swap_total
count,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,13195.000000,...,13195.000000,13195.000000,1.319500e+04,13195.0,1.319500e+04,1.319500e+04,13195.0,13195.0,13195.0,13195.0
mean,22.560967,0.051037,22.841455,22.258067,31.190676,0.072317,31.388708,30.985449,31.562257,0.066015,...,2068.385828,2064.011974,4.182962e+06,0.0,4.182962e+06,4.182962e+06,4194240.0,0.0,4194240.0,4194240.0
std,2.596492,0.195693,2.544142,2.651237,2.382058,0.114362,2.372173,2.413098,2.418098,0.127191,...,173.265695,171.689768,2.903330e+04,0.0,2.903330e+04,2.903330e+04,0.0,0.0,0.0,0.0
min,12.960000,0.000000,13.200000,12.200000,22.000000,0.000000,22.000000,22.000000,22.600000,0.000000,...,1281.000000,1266.000000,4.108224e+06,0.0,4.108224e+06,4.108224e+06,4194240.0,0.0,4194240.0,4194240.0
25%,21.440000,0.012571,21.800000,21.000000,30.000000,0.000000,30.000000,30.000000,30.600000,0.000000,...,2030.000000,2027.000000,4.194240e+06,0.0,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0
50%,23.280000,0.025524,23.600000,23.000000,32.000000,0.000000,32.000000,32.000000,32.000000,0.000000,...,2105.000000,2101.000000,4.194240e+06,0.0,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0
75%,24.400000,0.054095,24.600000,24.200000,33.000000,0.171429,33.000000,33.000000,33.000000,0.123810,...,2152.000000,2148.000000,4.194240e+06,0.0,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0
max,26.733333,19.174095,26.800000,26.600000,36.000000,4.923810,36.000000,36.000000,37.933333,7.600000,...,4149.000000,4138.000000,4.194240e+06,0.0,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0


In [9]:
lab.describe()

count    13195.000000
mean         0.032891
std          0.254373
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: New_label, dtype: float64